### Бизнес-кейс по прогнозированию трафика ( 4 часа )
Требуется
 1. Создать модель, которая прогнозирует количество чеков по дням в точке ( входные статистические данные приложены в csv формате ) 
 2. Оценить качество модели
 3. Написать перечень вопросов, которые возникли во время выполнения кейса 
 4. Накидать план разработки модели ( что не было сделано из-за нехватки времени)
  
Результат теста должен быть 
 1. скрипт результатом является с прогнозом на 1 месяц  (таблица результата – точка, день, количество чеков)
 2. минипрезентаця/миниотчет с результатами и выводами, замечаниями, вопросами

Данные:
Основные поля 
- point_code — код точки (магазина) 
- day — дата продажи 
- hour — час продажи 
- num_of_buyers — количество покупателей 

Не основные поля (дополненные самостоятельно) 
- is_pre_holiday — является ли день предпраздничным 
- is_common_holiday — является ли день национальным праздником (например 23 февраля, 8 марта) 
- is_working_day — является ли день рабочим 
- is_dayoff — является ли день выходным 
- is_limited_alcohol_sale — ограничена ли продажа алкоголя в этот день 
- is_local_holiday — является ли день локальным праздником (например день города) 
- local_holiday_distance_kilometers — расстояние от точки до близлежащего места проведения события, в километрах

In [147]:
import numpy as np
import pandas as pd

In [148]:
df = pd.read_csv('../data/result.csv', parse_dates=[1])

C:\Users\avvoy\AppData\Local\Temp\ipykernel_18544\3980290695.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/result.csv', parse_dates=[1])


# Предварительное изучение данных

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2485863 entries, 0 to 2485862
Data columns (total 11 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   point_code                         object        
 1   day                                datetime64[ns]
 2   hour                               int64         
 3   num_of_buyers                      int64         
 4   is_pre_holiday                     int64         
 5   is_common_holiday                  int64         
 6   is_working_day                     int64         
 7   is_dayoff                          int64         
 8   is_limited_alcohol_sale            int64         
 9   is_local_holiday                   int64         
 10  local_holiday_distance_kilometers  object        
dtypes: datetime64[ns](1), int64(8), object(2)
memory usage: 208.6+ MB


In [150]:
df.isna().sum()

point_code                                 0
day                                        0
hour                                       0
num_of_buyers                              0
is_pre_holiday                             0
is_common_holiday                          0
is_working_day                             0
is_dayoff                                  0
is_limited_alcohol_sale                    0
is_local_holiday                           0
local_holiday_distance_kilometers    2485127
dtype: int64

Очень много пропусков в последнем признаке, не использую его, он бесполезен

In [151]:
df.drop(columns='local_holiday_distance_kilometers', inplace=True)

Явные пропуски есть только в признаке local_holiday_distance_kilometers, который связан с предыдущим признаком is_local_holiday.

In [152]:
df['point_code'].describe()

count     2485863
unique        250
top          M634
freq        15364
Name: point_code, dtype: object

Предоставлена информация по 250 уникальным точкам сети.
Больше всего инфомации по точкам М, в частности `M634`, там 15364 строки данных.

Это самые старые, московские точки сети.

In [153]:
df['point_code'].value_counts()

point_code
M634    15364
M662    15364
M637    15364
M635    15364
M632    15364
        ...  
O566      828
O570      437
O554      368
O569      230
O583      115
Name: count, Length: 250, dtype: int64

Есть точки где накоплено совсем незначительное количество данных.
Вероятно недавно открытые точки (необходимо изучить временные метки).

По кодам точек деляю предположении о том что:
- первая литера отвечает за название города
- цифры после номер точки в городе в хронологическом порядке

### Обработка даты
исследование данных в столбце hour

In [154]:
df['hour'].nunique()

23

In [155]:
df['hour'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23], dtype=int64)

Хм, вижу некооректность в данных. Должно быть 24 часа. Есть только 23. По логике метки отвечают за начала часа. 

Т.е. 1 - это час который начинается в 1 час ночи и заканчивается в 2.
Т.е. 23 - это час который начинается в 23 часа вечера заканчивается в полночь.

Вероятно был некорректный запрос или сбой.

В данные нужно будет вручную добавлять первый час суток с индексом 0.

In [156]:
# собираю дату в один столбец
df['DateTime'] = df['day'] + pd.to_timedelta(df['hour'], unit='h')

# удаляю исходные признаки
df.drop(columns=['day', 'hour'], inplace=True)

In [157]:
df['DateTime'].describe()

count                          2485863
mean     2022-02-03 20:19:45.510866432
min                2021-01-01 01:00:00
25%                2021-09-25 13:00:00
50%                2022-03-07 21:00:00
75%                2022-07-06 16:00:00
max                2022-10-23 23:00:00
Name: DateTime, dtype: object

В датасете представлены данные за период с 2021-01-01 по 2022-10-23

### Поиск дубликатов

In [158]:
df

,point_code,num_of_buyers,is_pre_holiday,is_common_holiday,is_working_day,is_dayoff,is_limited_alcohol_sale,is_local_holiday,DateTime
0,A013,0,0,1,0,1,0,0,2021-01-01 01:00:00
1,A013,0,0,1,0,1,0,0,2021-01-01 02:00:00
2,A013,0,0,1,0,1,0,0,2021-01-01 03:00:00
3,A013,0,0,1,0,1,0,0,2021-01-01 04:00:00
4,A013,0,0,1,0,1,0,0,2021-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...
2485858,O583,24,0,0,0,1,0,0,2022-10-23 19:00:00
2485859,O583,26,0,0,0,1,0,0,2022-10-23 20:00:00
2485860,O583,18,0,0,0,1,0,0,2022-10-23 21:00:00
2485861,O583,12,0,0,0,1,0,0,2022-10-23 22:00:00


In [159]:
duplicated_rows = df[df.duplicated(keep=False)].sort_values(by=['point_code','DateTime'], ascending=False)
duplicated_rows.head(20)

,point_code,num_of_buyers,is_pre_holiday,is_common_holiday,is_working_day,is_dayoff,is_limited_alcohol_sale,is_local_holiday,DateTime
2484551,O566,0,0,0,0,1,0,0,2022-10-01 23:00:00
2484574,O566,0,0,0,0,1,0,0,2022-10-01 23:00:00
2484550,O566,11,0,0,0,1,0,0,2022-10-01 22:00:00
2484573,O566,11,0,0,0,1,0,0,2022-10-01 22:00:00
2484549,O566,7,0,0,0,1,0,0,2022-10-01 21:00:00
2484572,O566,7,0,0,0,1,0,0,2022-10-01 21:00:00
2484548,O566,7,0,0,0,1,0,0,2022-10-01 20:00:00
2484571,O566,7,0,0,0,1,0,0,2022-10-01 20:00:00
2484547,O566,14,0,0,0,1,0,0,2022-10-01 19:00:00
2484570,O566,14,0,0,0,1,0,0,2022-10-01 19:00:00


По всей видимости имеем дублирование некторых записей.

Визуальный анализ показал идентичность записей.

Дубликаты просто удаляю.

In [160]:
df.drop_duplicates(inplace=True, ignore_index=True)

In [161]:
df

,point_code,num_of_buyers,is_pre_holiday,is_common_holiday,is_working_day,is_dayoff,is_limited_alcohol_sale,is_local_holiday,DateTime
0,A013,0,0,1,0,1,0,0,2021-01-01 01:00:00
1,A013,0,0,1,0,1,0,0,2021-01-01 02:00:00
2,A013,0,0,1,0,1,0,0,2021-01-01 03:00:00
3,A013,0,0,1,0,1,0,0,2021-01-01 04:00:00
4,A013,0,0,1,0,1,0,0,2021-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...
2455107,O583,24,0,0,0,1,0,0,2022-10-23 19:00:00
2455108,O583,26,0,0,0,1,0,0,2022-10-23 20:00:00
2455109,O583,18,0,0,0,1,0,0,2022-10-23 21:00:00
2455110,O583,12,0,0,0,1,0,0,2022-10-23 22:00:00


### Необходимо разделение даных на датасеты по отдельным точкам

In [162]:
def get_point_by_code(df, code):
    tmp_df = df[df['point_code'] == code].copy()
    return tmp_df.drop(columns='point_code').reset_index(drop=True)

In [163]:
# словарь для храннения датасетов по точкам где в ключах имя точки, в значение датафрейм
points = {}

for code in df['point_code'].unique():
    points[code] = get_point_by_code(df, code)

Для дальнейшего комплексного исследования возьму одну из точек с наибольшим количеством данных M634

In [164]:
POINT = 'M634'
point_df = points[POINT]
point_df

,num_of_buyers,is_pre_holiday,is_common_holiday,is_working_day,is_dayoff,is_limited_alcohol_sale,is_local_holiday,DateTime
0,0,0,1,0,1,0,0,2021-01-01 01:00:00
1,0,0,1,0,1,0,0,2021-01-01 02:00:00
2,0,0,1,0,1,0,0,2021-01-01 03:00:00
3,0,0,1,0,1,0,0,2021-01-01 04:00:00
4,0,0,1,0,1,0,0,2021-01-01 05:00:00
...,...,...,...,...,...,...,...,...
15198,34,0,0,0,1,0,0,2022-10-23 19:00:00
15199,26,0,0,0,1,0,0,2022-10-23 20:00:00
15200,32,0,0,0,1,0,0,2022-10-23 21:00:00
15201,15,0,0,0,1,0,0,2022-10-23 22:00:00


Трансформация данных в полноценный временной ряд

Добввление пропущенных часов, которые заполняются в обратном порядке. В даннмо случае это логично, пропущены первые часы суток, где продаж нет магазины ночью не работают.

In [171]:
def transform_to_ts(df):
    new_df = df.copy()
    new_df.set_index("DateTime", inplace=True)
    full_date_range = pd.date_range(
        start=new_df.index.min().replace(hour=0, minute=0, second=0, microsecond=0), end=new_df.index.max(), freq="H"
    )
    new_df = new_df.reindex(full_date_range).bfill()
    return new_df.astype(int)

In [172]:
point_ts = transform_to_ts(point_df)

In [173]:
point_ts

,num_of_buyers,is_pre_holiday,is_common_holiday,is_working_day,is_dayoff,is_limited_alcohol_sale,is_local_holiday
2021-01-01 00:00:00,0,0,1,0,1,0,0
2021-01-01 01:00:00,0,0,1,0,1,0,0
2021-01-01 02:00:00,0,0,1,0,1,0,0
2021-01-01 03:00:00,0,0,1,0,1,0,0
2021-01-01 04:00:00,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...
2022-10-23 19:00:00,34,0,0,0,1,0,0
2022-10-23 20:00:00,26,0,0,0,1,0,0
2022-10-23 21:00:00,32,0,0,0,1,0,0
2022-10-23 22:00:00,15,0,0,0,1,0,0


### В задаче говорится о прогнозировании дневного количества чеков, сжимаю данные

In [175]:
point_ts_d = point_ts.resample('D').agg({'num_of_buyers': 'sum', 
                                         'is_pre_holiday': 'first',
                                         'is_common_holiday': 'first',
                                         'is_working_day': 'first',
                                         'is_dayoff': 'first',
                                         'is_limited_alcohol_sale': 'first',
                                         'is_local_holiday': 'first',
                                         })


In [ ]:
point_ts_d

,num_of_buyers,is_pre_holiday,is_common_holiday,is_working_day,is_dayoff,is_limited_alcohol_sale,is_local_holiday,local_holiday_distance_kilometers
2021-01-01,207.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
2021-01-02,296.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
2021-01-03,273.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
2021-01-04,260.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
2021-01-05,305.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
2022-10-19,380.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
2022-10-20,407.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
2022-10-21,448.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN
2022-10-22,416.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN


Сохраняю подготовленные временные ряды для EDA и дальнейшей обработки

In [176]:
point_ts_d.to_csv(f"../data/point_{POINT}_ts.csv")